In [ ]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import google.generativeai as genai
import datetime

today = datetime.date.today().isoformat()


# Percorso al file della chiave
api_key_path = os.path.join("config", "google_api")
try:
    with open(api_key_path, "r") as f:
        api_key = f.read().strip()
except FileNotFoundError:
    api_key = None

if not api_key:
    raise ValueError("La chiave API di Google non è stata trovata. Assicurati che il file 'google_api' esista nella cartella 'config'.")

genai.configure(api_key=api_key)

# Carica embedding e indice FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_index = FAISS.load_local(
    "../embeddings/faiss_index_hf",
    embedding_model,
    allow_dangerous_deserialization=True
)

def ask_gemini(query, top_k=5):
    risultati = faiss_index.similarity_search(query, k=top_k)
    if not risultati:
        return "Nessun documento rilevante trovato."

    contesto = "\n\n".join([r.page_content for r in risultati])
    prompt = f"""
Sei un assistente personale che ha a disposizione una lista di eventi con date di inizio e fine.
La data odierna è: {today}

La domanda è: {query}

Dai eventi forniti, rispondi elencando solo quelli che ricadono nel periodo di tempo indicato nella domanda, se la domanda lo richiede.
Se la domanda non specifica un periodo, rispondi con tutti gli eventi rilevanti.

Se non trovi eventi rilevanti, rispondi con "Nessun evento trovato nel periodo richiesto."

Ecco i dati disponibili (eventi):

{contesto}

Risposta:
"""


    modello = genai.GenerativeModel("gemini-1.5-flash")
    risposta = modello.generate_content(prompt)
    return risposta.text or "Il modello non ha generato una risposta."

def main():
    print("Chat interattiva con Gemini (digita 'exit' per uscire):")
    while True:
        domanda = input("\nTu: ")
        if domanda.strip().lower() in ["exit", "esci", "quit"]:
            print("Uscita. A presto!")
            break

        print("Gemini:", end=" ")
        try:
            risposta = ask_gemini(domanda)
            print(risposta)
        except Exception as e:
            print(f"\nErrore: {e}")

if __name__ == "__main__":
    main()


c:\Users\VincenzoColella\Documents\GitHub\RAG_Vincenzo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\VincenzoColella\AppData\Local\Temp\ipykernel_24012\707179182.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Chat interattiva con Gemini (digita 'exit' per uscire):
Gemini: Nessun evento trovato nel periodo richiesto.

Gemini: Nessun evento trovato nel periodo richiesto.

Gemini: Gli eventi dell'anno scorso (2024) sono:

* 2024-01-06 00:00 - 2024-01-07 00:00: Vau da PostFinance
* 2024-02-15 17:00 - 2024-02-15 18:00: Unghie
* 2024-04-27 15:00 - 2024-04-27 16:00: 5km de Lausanne alle 17


Gemini: Gli eventi di tre anni fa (rispetto al 24 Giugno 2025) ricadono nel periodo compreso tra il 24 Giugno 2022 e il 24 Giugno 2023.  Gli eventi di questo periodo sono:

* 2022-05-12 15:30 - 2022-05-12 16:30: Sapienza AI seminario
* 2022-06-27 00:00 - 2022-06-28 00:00: Genitori CR
* 2022-07-21 14:30 - 2022-07-21 15:30: Louvre
* 2023-03-11 07:00 - 2023-03-11 08:00: Padel con Adriano
* 2023-05-09 00:00 - 2023-05-10 00:00: Festa marghe
* 2023-07-09 00:00 - 2023-07-10 00:00: Concerto Coez


Gemini: Nessun evento trovato nel periodo richiesto.  I dati forniti sono informazioni personali e professionali, non even